# Tradução dos Códigos de Ocupação O*NET para ocupações do Brasil (COD 2010)

Para aplicar a metodologia de Dingel e Neiman, é necessário realizar a tradução dos códigos de ocupações para ocupações do Brasil.

**Passo-a-passo da tradução dos códigos de ocupação, segundo artigo da FGV:**

1. Primeiro, realizamos uma tradução das 968 ocupações da O*NET para a Standard Occupational Classification de 2010 (SOC-2010), utilizada na pesquisa por domicílios americana (Current Population Survey - CPS);
2. Em seguida, reclassificamos as ocupações da SOC-2010 para a ISCO-08, para então; 
3. traduzirmos para a COD utilizada na PNAD Contínua. 

As reclassificações foram realizadas conforme o diagrama abaixo.

#### Diagrama para tradução dos códigos de ocupação (FGV, 2021)
![Diagrama](../Anexos/diagrama_traducao.png)

Link para base de ocupações da O*NET: [O*NET OnLine](https://www.onetonline.org/find/all)

Link para base de ocupações da SOC-2010: [U.S. BUREAU OF LABOR STATISTICS](https://www.bls.gov/soc/2010/home.htm)

Link para base de ocupações da ISCO-08: [International Labour Organization](https://isco-ilo.netlify.app/en/isco-08/)

Repositório do estudo Work at Home (Dingel e Neiman): [Work at Home](https://github.com/jdingel)

Arquivo CSV com as ocupações teletrabalháveis: [occupations_workathome.csv](https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv)



In [173]:
#TODO: refazer o código do SOC 2010 para traduzir para código da ISCO-08

In [174]:
#pacotes
import pandas as pd
import numpy as np

In [175]:
cod2010

,0,Grandes Grupos,Subgrupos Principal,Subgrupos,Grupos de Base,Titulação
0,1,1,NaN,NaN,NaN,DIRETORES E GERENTES
1,11,1,11.0,NaN,NaN,"DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRA..."
2,111,1,11.0,111.0,NaN,MEMBROS SUPERIORES DO PODER EXECUTIVO E LEGISL...
3,1111,1,11.0,111.0,1111.0,LEGISLADORES
4,1112,1,11.0,111.0,1112.0,DIRIGENTES SUPERIORES DA ADMINISTRAÇÃO PÚBLICA
...,...,...,...,...,...,...
613,511,0,51.0,511.0,511.0,OFICIAIS DE BOMBEIRO MILITAR
614,512,0,51.0,512.0,512.0,GRADUADOS E PRAÇAS DO CORPO DE BOMBEIROS
615,599,0,59.0,599.0,599.0,BOMBEIROS E POLICIAIS MILITARES NÃO ESPECIFICADOS
616,999,0,99.0,999.0,999.0,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."


In [215]:
#Leitura das bases (tabelas de ocupação e de pessoas)

onet = pd.read_excel('../Anexos/Occupations_ONET.xlsx') #Ocupações O*NET
soc2010 = pd.read_excel('../Anexos/Occupations_SOC-2010.xlsx') #Ocupações SOC-2010
isco08 = pd.read_excel('../Anexos/Occupations_ISCO-08.xlsx') #Ocupações ISCO-08
#cod2010 = pd.read_excel('../Anexos/COD 2010.xlsx', ) #Ocupações COD 2010

cod2010 = pd.read_excel('../Anexos/COD 2010.xlsx', dtype={'Cod': str}) #Ocupações COD 2010

onet_teleworkable = pd.read_csv('https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv')

In [177]:
#Ocupações na base O*NET que podem ser realizadas em teletrabalho
onet_teleworkable

,onetsoccode,title,teleworkable
0,11-1011.00,Chief Executives,1
1,11-1011.03,Chief Sustainability Officers,1
2,11-1021.00,General and Operations Managers,1
3,11-2011.00,Advertising and Promotions Managers,1
4,11-2021.00,Marketing Managers,1
...,...,...,...
963,53-7072.00,"Pump Operators, Except Wellhead Pumpers",0
964,53-7073.00,Wellhead Pumpers,0
965,53-7081.00,Refuse and Recyclable Material Collectors,0
966,53-7111.00,Mine Shuttle Car Operators,0


### Tradução do O\*NET (filtrado por Dingel e Neiman) para o SOC-2010 

A tradução deve ser feita usando como refência o código do Major Group e título da ocupação

In [178]:
soc2010filtrado = soc2010[['Detailed Occupation', 'Title']]
soc2010filtrado.columns = ['code_SOC', 'Title_SOC']
soc2010filtrado.dropna(inplace=True)

soc2010['code_SOC'] = soc2010.iloc[:, :4].apply(lambda x: ''.join(x.dropna().astype(str)), axis=1)
#Captura somente o codigo major
soc2010filtrado['MAJOR_SOC'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])

C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_18092\1001432072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc2010filtrado.dropna(inplace=True)
C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_18092\1001432072.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc2010filtrado['MAJOR_SOC'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])


In [179]:
onet_filtrado = onet_teleworkable.copy()
onet_filtrado.columns = ['code_ONET', 'Title_ONET', 'teleworkable']

#captura somente o codigo major
onet_filtrado['code_ONET'] = onet_filtrado['code_ONET'].apply(lambda x: x[0:-3])

In [180]:
#join onet_filtrado e soc2010filtrado by column Code_ONET and code_SOC

onet_soc = pd.merge(onet_filtrado, soc2010filtrado, how='left', left_on='code_ONET', right_on='code_SOC')

Até aqui, ONET e SOC-2010 foram agregados com sucesso. Agora, vamos fazer o mesmo com ISCO-08 e COD-2010.

### Tradução de ISC08 para COD 2010

In [219]:
cod2010_filtrado = cod2010[['Cod', 'Titulação']]
cod2010_filtrado.columns = ['code_COD', 'Title_COD']

#Captura somente o codigo major
#cod2010_filtrado['MAJOR_cod'] = soc2010filtrado['code_SOC'].apply(lambda x: x[:2])

cod2010_filtrado['code_COD'] = cod2010_filtrado['code_COD'].astype(str)

C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_18092\481978680.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cod2010_filtrado['code_COD'] = cod2010_filtrado['code_COD'].astype(str)


In [220]:
isco08_filtrado = isco08[['ISCO 08 Code', 'Major', 'Title EN']]
isco08_filtrado.columns = ['code_ISCO', 'Major', 'Title_ISCO']

isco08_filtrado['code_ISCO'] = isco08_filtrado['code_ISCO'].astype(str) 
isco08_filtrado['Major'] = isco08_filtrado['Major'].astype(str) 

C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_18092\3149538693.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isco08_filtrado['code_ISCO'] = isco08_filtrado['code_ISCO'].astype(str)
C:\Users\est.matheus.brasil\AppData\Local\Temp\ipykernel_18092\3149538693.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isco08_filtrado['Major'] = isco08_filtrado['Major'].astype(str)


In [221]:
isco_cod = pd.merge(isco08_filtrado, cod2010_filtrado, how='left', left_on='Major', right_on='code_COD')

Até aqui, temos ISCO-08 e COD 2010 agregados. Agora, vamos fazer o join com as ocupações agregadas.

### Tradução de SOC-2010 para ISCO-08

In [223]:
ocupacoes_agregado = pd.merge(onet_soc, isco_cod, how='left', left_on='MAJOR_SOC', right_on='code_ISCO')

In [ ]:
#ocupacoes_agregado.fillna('OUTROS', inplace=True)

In [230]:
isco08

,Level,ISCO 08 Code,Major,Title EN,Definition,Tasks include,Included occupations,Excluded occupations,Notes
0,1,1,1,Managers,"Managers plan, direct, coordinate and evaluate...",Tasks performed by managers usually include: f...,Occupations in this major group are classified...,NaN,In distinguishing between managers classified ...
1,2,11,1,"Chief Executives, Senior Officials and Legisla...","Chief executives, senior officials and legisla...",Tasks performed by workers in this sub-major g...,Occupations in this sub-major group are classi...,NaN,NaN
2,3,111,1,Legislators and Senior Officials,"Legislators and senior officials determine, fo...",Tasks performed usually include: presiding ove...,Occupations in this minor group are classified...,NaN,NaN
3,4,1111,1,Legislators,"Legislators determine, formulate, and direct p...",Tasks include -\n(a) presiding over or partic...,Examples of the occupations classified here:\n...,NaN,NaN
4,4,1112,1,Senior Government Officials,Senior government officials advise governments...,"Tasks include -\n(a) advising national, state...",Examples of the occupations classified here:\n...,NaN,Chief executives of Government-owned enterpris...
...,...,...,...,...,...,...,...,...,...
614,3,21,0,Non-commissioned Armed Forces Officers,Non-commissioned armed forces officers enforce...,NaN,Occupations in this minor group are classified...,Excluded from this group are: \n - jobs held ...,NaN
615,4,210,0,Non-commissioned Armed Forces Officers,Non-commissioned armed forces officers enforce...,NaN,Examples of the occupations classified here:\n...,Excluded from this group are: \n - jobs held ...,NaN
616,2,3,0,"Armed Forces Occupations, Other Ranks","Armed forces occupations, other ranks include ...",NaN,Occupations in this sub-major group are classi...,Excluded from this group are: \n - jobs held ...,NaN
617,3,31,0,"Armed Forces Occupations, Other Ranks","Armed forces occupations, other ranks include ...",NaN,Occupations in this minor group are classified...,"parné\ndinero, posibles, cuartos, guita, mosca...",NaN


In [228]:
cod2010

,Cod,Grandes Grupos,Subgrupos Principal,Subgrupos,Grupos de Base,Titulação
0,1,1,NaN,NaN,NaN,DIRETORES E GERENTES
1,11,1,11.0,NaN,NaN,"DIRETORES EXECUTIVOS, DIRIGENTES DA ADMINISTRA..."
2,111,1,11.0,111.0,NaN,MEMBROS SUPERIORES DO PODER EXECUTIVO E LEGISL...
3,1111,1,11.0,111.0,1111.0,LEGISLADORES
4,1112,1,11.0,111.0,1112.0,DIRIGENTES SUPERIORES DA ADMINISTRAÇÃO PÚBLICA
...,...,...,...,...,...,...
613,0511,0,5.0,51.0,511.0,OFICIAIS DE BOMBEIRO MILITAR
614,0512,0,5.0,51.0,512.0,GRADUADOS E PRAÇAS DO CORPO DE BOMBEIROS
615,0599,0,5.0,59.0,599.0,BOMBEIROS E POLICIAIS MILITARES NÃO ESPECIFICADOS
616,0999,0,9.0,99.0,999.0,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."


#### trabalhos teletrabalháveis

In [224]:
ocupacoes_agregado['code_COD'].value_counts()

5    164
1    105
0     88
4     87
2     83
3     63
Name: code_COD, dtype: int64

In [208]:
cod2010['Grandes Grupos'].value_counts()

2    126
3    110
7     86
5     59
8     58
9     51
1     47
4     42
6     20
0     19
Name: Grandes Grupos, dtype: int64

In [211]:
ocupacoes_agregado.groupby('Title_COD')['code_COD'].count()

Title_COD
DIRETORES E GERENTES                                               105
MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEIROS MILITARES         88
PROFISSIONAIS DAS CIÊNCIAS E INTELECTUAIS                           83
TRABALHADORES DE APOIO ADMINISTRATIVO                               87
TRABALHADORES DOS SERVIÇOS, VENDEDORES DOS COMÉRCIOS E MERCADOS    164
TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO                             63
Name: code_COD, dtype: int64